## Saving 1 frame.


## Setting position for 320*320

In [9]:
import cv2
import os

# --- CONFIGURATION ---
video_path      = '/home/holidayj/Videos/videos/platform/chungmuro/chungmuro_hasun_20221019T172940_20221019T203040/chungmuro_hasun_20221019T172940_20221019T203040.mp4'
output_folder   = 'output_frames'
output_filename = 'chungmuro_cropped_320.jpg'
# crop_size       = 600
crop_size       = 700

# Cropping Margins
margin_top   = 30    # Move down pixels from the top edge
margin_right = 400  # Move left 120 pixels from the right edge
# ---------------------

os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video at {video_path}")
else:
    # 1. Search for the first valid frame (Fix for the [h264] error)
    frame_found = False
    max_attempts = 100
    
    print("Searching for a valid Keyframe...")
    
    for i in range(max_attempts):
        ret, frame = cap.read()
        if ret:
            print(f"Success: Valid frame found at index {i}")
            
            # --- CROP LOGIC STARTS HERE ---
            
            # 2. Get Dimensions
            height, width, _ = frame.shape
            
            # 3. Calculate Coordinates
            # Y: Start at top margin
            y_start = margin_top
            y_end = y_start + crop_size

            # X: Start from right side (width) - margin - crop_size
            x_end = width - margin_right
            x_start = x_end - crop_size

            print(f"Original Resolution: {width}x{height}")
            print(f"Cropping Area -> X: {x_start} to {x_end}, Y: {y_start} to {y_end}")

            # 4. Perform Crop
            cropped_frame = frame[y_start:y_end, x_start:x_end]

            # 5. Save
            full_save_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(full_save_path, cropped_frame)
            print(f"Saved cropped image to: {full_save_path}")
            
            frame_found = True
            break # Stop after saving the first valid frame
            
            # --- CROP LOGIC ENDS HERE ---

    if not frame_found:
        print("Error: Could not find any valid frames in the beginning of the video.")

cap.release()

Searching for a valid Keyframe...
Success: Valid frame found at index 1
Original Resolution: 1920x1080
Cropping Area -> X: 820 to 1520, Y: 30 to 730
Saved cropped image to: output_frames/chungmuro_cropped_320.jpg


[h264 @ 0xe6b6e40] missing picture in access unit with size 40
[h264 @ 0xe6b6e40] no frame!
[h264 @ 0xe696c40] no frame!


## Extracting frames

In [ ]:
'''
# chungmuro hasun config.
VIDEO_PATH    = '/home/holidayj/Videos/videos/platform/chungmuro/chungmuro_hasun_20221019T172940_20221019T203040/chungmuro_hasun_20221019T172940_20221019T203040.mp4'
OUTPUT_FOLDER = 'Python/annotation/frames/chungmuro_hasun'
INTERVAL_SEC  = 0.2
CROP_SIZE     = 600
MARGIN_RIGHT  = 400
MARGIN_TOP    = 10
'''
import cv2
import numpy as np
import os
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# --- CONFIGURATION ---
# VIDEO_PATH    = '/home/holidayj/Videos/videos/platform/chungmuro/chungmuro_hasun_20221019T172940_20221019T203040/chungmuro_hasun_20221019T172940_20221019T203040.mp4'
VIDEO_PATH    = '/home/holidayj/Videos/videos/platform/chungmuro/chungmuro_sangsun_20221019T172940-20221019T202940/chungmuro_sangsun_20221019T172940-20221019T202940.mp4'

OUTPUT_FOLDER = 'frames/chungmuro_sangsun_10frames_1920'
INTERVAL_SEC  = 1/3
CROP_SIZE     = 700
MARGIN_RIGHT  = 400
MARGIN_TOP    = 30



# OUTPUT_FOLDER = 'frames/chungmuro_hasun_6frames_700'
# INTERVAL_SEC  = 0.2
# CROP_SIZE     = 700
# MARGIN_RIGHT  = 400
# MARGIN_TOP    = 30
# ---------------------

def save_image_worker(args):
    """
    Independent worker function to save the image.
    This runs on separate CPUs.
    """
    img_data, save_path = args
    cv2.imwrite(save_path, img_data)

def main():
    # 1. Setup
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    # 2. Metadata
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print("fps =", np.round(fps))
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    # 3. Crop Config
    x_start = width - MARGIN_RIGHT - CROP_SIZE
    y_start = MARGIN_TOP
    frame_step = int(fps * INTERVAL_SEC)
    if frame_step < 1: frame_step = 1

    print(f"FPS: {fps} | Step: {frame_step}")
    print(f"Using {cpu_count()} CPUs for saving images.")

    # 4. Initialize the Worker Pool (For saving only)
    # We use roughly 80% of CPUs to leave room for the main reader process
    worker_count = max(1, cpu_count() - 1) 
    pool = Pool(processes=worker_count)
    
    current_idx = 0
    saved_count = 0

    # 5. Fast Reader Loop
    # The main loop now NEVER waits for disk I/O. 
    # It just throws the image to the pool and immediately reads the next one.
    with tqdm(total=total_frames, unit="frame") as pbar:
        while True:
            ret, frame = cap.read()

            if not ret:
                if current_idx < 100: # Skip initial corruption
                    current_idx += 1
                    pbar.update(1)
                    continue
                else:
                    break

            if current_idx % frame_step == 0:
                # Crop
                cropped = frame[0:1080,
                                0:1920]
                # cropped = frame[y_start : y_start + CROP_SIZE, 
                #                 x_start : x_start + CROP_SIZE]
                
                # Construct path
                filename = f"chungmuro_frame_{current_idx:06d}.jpg"
                save_path = os.path.join(OUTPUT_FOLDER, filename)
                
                # --- ASYNC SAVE ---
                # Fire and forget. The main loop continues immediately.
                pool.apply_async(save_image_worker, args=((cropped, save_path),))
                saved_count += 1

            current_idx += 1
            pbar.update(1)

    cap.release()
    
    print("\nReading finished. Waiting for remaining file writes to complete...")
    pool.close()
    pool.join() # Wait for the background workers to finish saving
    print(f"Done! Saved {saved_count} images.")

if __name__ == '__main__':
    main()

fps = 30.0


[h264 @ 0x22a4c880] missing picture in access unit with size 40
[h264 @ 0x22a4c880] no frame!


# Counting class

In [5]:
import os
import glob
from collections import defaultdict

# Define the path to your dataset
dataset_path = '/media/holidayj/Documents/data/euljiro_2nd/with_descending'

# Find all .txt files in the directory
label_files = glob.glob(os.path.join(dataset_path, '*.txt'))

print(f"Found {len(label_files)} label files in {dataset_path}")

# Initialize a dictionary to count objects per class
class_counts = defaultdict(int)

# Iterate through each label file
for file_path in label_files:
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                # Ensure the line is not empty
                if parts:
                    # In YOLO format, the first element is the class ID
                    class_id = int(parts[0])
                    class_counts[class_id] += 1
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Print the results
print("\nObject counts per class:")
# Sort by class ID for cleaner output
for class_id in sorted(class_counts.keys()):
    print(f"Class {class_id}: {class_counts[class_id]}")

Found 2741 label files in /media/holidayj/Documents/data/euljiro_2nd/with_descending
Error reading /media/holidayj/Documents/data/euljiro_2nd/with_descending/classes.txt: invalid literal for int() with base 10: 'U'

Object counts per class:
Class 0: 2600
Class 1: 3473
Class 2: 5312


In [2]:
import os
import shutil
import glob

# --- Configuration ---
source_dir = '/media/holidayj/Documents/data/euljiro_2nd'
target_folder_name = 'without_descending'
target_dir = os.path.join(source_dir, target_folder_name)
target_class = 1  # The class ID for "descending"

# --- Setup ---
# Create the destination directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)
    print(f"Created directory: {target_dir}")

# Get list of all label files
label_files = glob.glob(os.path.join(source_dir, '*.txt'))
moved_count = 0

print(f"Scanning {len(label_files)} files...")

# --- Processing ---
for label_path in label_files:
    filename = os.path.basename(label_path)
    file_base_name = os.path.splitext(filename)[0]
    
    has_descending = False
    
    try:
        with open(label_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 0:
                    class_id = int(parts[0])
                    if class_id == target_class:
                        has_descending = True
                        break  # Stop checking this file if we found class 1
        
        # If NO descending class was found, move the files
        if not has_descending:
            # 1. Move the Label file
            shutil.move(label_path, os.path.join(target_dir, filename))
            
            # 2. Find and Move the Image file
            # We check common image extensions
            image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
            image_found = False
            
            for ext in image_extensions:
                image_name = file_base_name + ext
                src_image_path = os.path.join(source_dir, image_name)
                
                if os.path.exists(src_image_path):
                    shutil.move(src_image_path, os.path.join(target_dir, image_name))
                    image_found = True
                    break # Stop checking extensions once image is found
            
            moved_count += 1
            if not image_found:
                print(f"Warning: Moved label {filename}, but could not find corresponding image.")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

print("---")
print(f"Process complete. Moved {moved_count} pairs to '{target_dir}'.")

Scanning 5072 files...
Error processing classes.txt: invalid literal for int() with base 10: 'U'
---
Process complete. Moved 2533 pairs to '/media/holidayj/Documents/data/euljiro_2nd/without_descending'.


In [6]:
import os
import shutil

# Define the directories
# Where the JPGs are currently located (and where TXTs should go)
target_dir = '/media/holidayj/Documents/data/euljiro_2nd/without_descending/descent'

# Where the TXTs are currently located
source_txt_dir = '/media/holidayj/Documents/data/euljiro_2nd/without_descending'

# Counter to track progress
count = 0

# Iterate through all files in the target directory (the descent folder)
for filename in os.listdir(target_dir):
    # Check if the file is a JPG
    if filename.lower().endswith('.jpg'):
        
        # Extract the filename without the extension (e.g., 'image_01')
        file_root = os.path.splitext(filename)[0]
        
        # Construct the expected text file name
        txt_filename = file_root + '.txt'
        
        # Define the full path for the source text file
        src_txt_path = os.path.join(source_txt_dir, txt_filename)
        
        # Define the full destination path
        dst_txt_path = os.path.join(target_dir, txt_filename)
        
        # Check if the corresponding text file exists in the source directory
        if os.path.exists(src_txt_path):
            try:
                # Move the file
                shutil.move(src_txt_path, dst_txt_path)
                print(f"Matched and Moved: {txt_filename}")
                count += 1
            except Exception as e:
                print(f"Error moving {txt_filename}: {e}")

print(f"---")
print(f"Operation Complete. Total files moved: {count}")

Matched and Moved: euljiro_frame_247505.txt
Matched and Moved: euljiro_frame_183903.txt
Matched and Moved: euljiro_frame_124608.txt
Matched and Moved: euljiro_frame_189567.txt
Matched and Moved: euljiro_frame_269866.txt
Matched and Moved: euljiro_frame_151276.txt
Matched and Moved: euljiro_frame_074753.txt
Matched and Moved: euljiro_frame_246266.txt
Matched and Moved: euljiro_frame_256178.txt
Matched and Moved: euljiro_frame_217415.txt
Matched and Moved: euljiro_frame_240543.txt
Matched and Moved: euljiro_frame_214760.txt
Matched and Moved: euljiro_frame_330577.txt
Matched and Moved: euljiro_frame_067142.txt
Matched and Moved: euljiro_frame_328217.txt
Matched and Moved: euljiro_frame_293938.txt
Matched and Moved: euljiro_frame_086317.txt
Matched and Moved: euljiro_frame_154639.txt
Matched and Moved: euljiro_frame_207267.txt
Matched and Moved: euljiro_frame_086907.txt
Matched and Moved: euljiro_frame_295944.txt
Matched and Moved: euljiro_frame_266621.txt
Matched and Moved: euljiro_frame

In [11]:
import cv2
import os
import glob
from tqdm import tqdm  # Progress bar (optional, install with `pip install tqdm`)

# 1. Define paths
input_folder = '/media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_60frames_700'
output_folder = os.path.join(input_folder, 'equalized')

# 2. Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# 3. Get list of all jpg images
image_files = glob.glob(os.path.join(input_folder, '*.jpg'))
print(f"Found {len(image_files)} images.")

# 4. Process images
for img_path in tqdm(image_files, desc="Processing"):
    # Read the image
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to read: {img_path}")
        continue

    # Convert from BGR to YCrCb (we want to equalize luminance 'Y', not colors)
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    # Equalize the histogram of the Y channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    # Convert back to BGR
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YCrCb2BGR)

    # Save the result
    filename = os.path.basename(img_path)
    save_path = os.path.join(output_folder, filename)
    cv2.imwrite(save_path, img_output)

print(f"\nProcessing complete! \nImages saved to: {output_folder}")

Found 5430 images.


Processing: 100%|██████████| 5430/5430 [03:13<00:00, 28.09it/s]


Processing complete! 
Images saved to: /media/holidayj/Documents/github/ML/Python/annotation/frames/chungmuro_hasun_60frames_700/equalized
